In [ ]:
%matplotlib inline



# Get reduced matrices and make export
This example shows how to get reduced matrices and
export them to hdf5 and csv format.


Import dpf module and its examples files, and create a temporary directory



In [ ]:
import os
import tempfile

from ansys.dpf import core as dpf
from ansys.dpf.core import examples
from ansys.dpf.core import operators as ops
from ansys.jupyterhub.manager import spawn_dpf, delete_pod

tmpdir = tempfile.mkdtemp()

Create the operator and connect dataSources



In [ ]:
server, pod_name = spawn_dpf()
local_file = examples.download_sub_file()
server_file_path = dpf.upload_file_in_tmp_folder(local_file, server=server)
ds = dpf.DataSources(server_file_path, server=server)

matrices_provider = ops.result.cms_matrices_provider()
matrices_provider.inputs.data_sources.connect(ds)

Get result fields container that contains the reduced matrices



In [ ]:
fields = matrices_provider.outputs.fields_container()

len(fields)

fields[0].data

Export the result fields container in hdf5 format



In [ ]:
h5_op = ops.serialization.serialize_to_hdf5()
h5_op.inputs.data1.connect(matrices_provider.outputs)
h5_op.inputs.file_path.connect("matrices.h5")
h5_op.run()

Download the hdf5 file from the server

In [ ]:
dpf.download_file("matrices.h5", os.path.join(tmpdir, "matrices.h5"), server=server)

Export the result fields container in csv format



In [ ]:
csv_op = ops.serialization.field_to_csv()
csv_op.inputs.field_or_fields_container.connect(matrices_provider.outputs)
csv_op.inputs.file_path.connect("matrices.csv")
csv_op.run()

Download the csv file from the server

In [ ]:
dpf.download_file("matrices.csv", os.path.join(tmpdir, "matrices.csv"), server=server)

In [ ]:
delete_pod(pod_name)